# Vectorization

The aim of this notebook is to produce different set of vectorized tasting descriptions. These vectorized sets of tasting descriptions will then either be directly used to compute distance between description and recommend wines, or fed to a latent semantic analysis (topic modeling).


In [ ]:
# general:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from collections import Counter

from sklearn.pipeline import make_pipeline

# NLP:
import spacy

In [2]:
# load data:
winedata = pd.read_csv("../data/winedata_parsed.csv")
print(winedata.shape)
winedata.head()

(141540, 15)


,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,vintage,description_parsed
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013.0,"['aromas', 'include', 'tropical', 'fruit', 'br..."
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011.0,"['ripe', 'fruity', 'wine', 'smooth', 'structur..."
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013.0,"['tart', 'snappy', 'flavors', 'lime', 'flesh',..."
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013.0,"['pineapple', 'rind', 'lemon', 'pith', 'orange..."
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012.0,"['like', 'regular', 'bottling', '2012', 'comes..."


## Bag of Words (BoW)

Build a set of features using the BoW technique.

In [11]:
# Construct full corpus, by joing all the words in the parsed description
# in a big list:     !!!! TAKE A LONG TIME TO RUN !!!!
from ast import literal_eval
wine_corpus = winedata['description_parsed'].apply(literal_eval).sum()

In [16]:
print('total number words in corpus: ', len(wine_corpus))

# construct a list of unique words in the corpus:
all_wine_tasting_words = np.unique(wine_corpus)
print('number of unique words in corpus: ', len(all_wine_tasting_words))
print('excerpt:')
all_wine_tasting_words[0:200]

total number words in corpus:  3433627
number of unique words in corpus:  33974
excerpt:


array(['\r\n', ' ', '  ', '   ', '        ', '$', "'em", '+', '-2043',
       '-aged', '-and', '-berry', '-bodied', '-cherry', '-colored',
       '-composed', '-cranberry', '-driven', '-elegant', '-fruit',
       '-inflected', '-inhabited', '-j.m.', '-label', '-laced',
       '-lavored', '-m.s.', '-pepper', '-piquant', '-plum', '-raspberry',
       '-ruit', '-run', '-salt', '-savory', '-syrah', '-tone',
       '-vineyard', '-weet', '-year', '.01', '.82', '.95', '.a', '.this',
       '0-degree', '0.4', '0.5', '0.53', '0.6', '0.7', '0.8', '01', '02',
       '03', '04', '04s', '04—which', '05', '05s', '06', '06s', '07',
       '07s', '08', '08/11/99', '08s', '09', '09s', '1', '1%touriga',
       '1,000', '1,000-acre', '1,000-feet', '1,000-foot', '1,000-liter',
       '1,000–1,400-foot', '1,030-feet', '1,100', '1,100-feet',
       '1,100–1,200', '1,110-feet', '1,150', '1,200', '1,200-feet',
       '1,200-foot', '1,250', '1,275', '1,280', '1,300', '1,300-feet',
       '1,300-foot', '1,300–1

We see here that there are words/numbers/punctuation marks that will not be useful. We could clean the text more carefully at the previous step. Nevertheless, we keep them for now, as those will not be among the most common words anyway. Let's have a look at another excerpt:

In [17]:
all_wine_tasting_words[2000:2200]

array(['adjacent', 'adjective', 'adjectives', 'adjoining', 'adjoins',
       'adjunct', 'adjusted', 'adjustment', 'adjusts', 'administered',
       'admirable', 'admirably', 'admiral', 'admiration', 'admire',
       'admired', 'admirers', 'admission', 'admit', 'admits',
       'admittance', 'admitted', 'admittedly', 'admitting', 'admixture',
       'ado', 'adobe', 'adolescence', 'adolescent', 'adolph', 'adopted',
       'adopting', 'adopts', 'adorably', 'adore', 'adored', 'adoring',
       'adorn', 'adorned', 'adorning', 'adornments', 'adorns', 'adour',
       'adquate', 'adriana', 'adrianna', 'adriano', 'adriatic',
       'adrienne', 'adroitly', 'adult', 'adults', 'advance', 'advanced',
       'advancement', 'advances', 'advantage', 'adventure', 'adventurer',
       'adventurers', 'adventures', 'adventurous', 'adversely',
       'advertised', 'advertisement', 'advertises', 'advertising',
       'advice', 'advisable', 'advised', 'advocacy', 'advocate',
       'adènzia', 'adèo', 'aegean

In [19]:
all_wine_tasting_words[33000:33974]

array(['walnuts', 'walnutty', 'walpole', 'walsh', 'walter', 'waltz',
       'waltzes', 'wamrth', 'wan', 'wander', 'wanders', 'wane', 'wanes',
       'waning', 'wannabe', 'wannabes', 'want', 'wanted', 'wanting',
       'wants', 'war', 'warboys', 'warehouse', 'warm', 'warmed', 'warmer',
       'warmest', 'warmhearted', 'warming', 'warmish', 'warmly', 'warms',
       'warmth', 'warn', 'warned', 'warner', 'warning', 'warns',
       'warramate', 'warrant', 'warranted', 'warrants', 'warre', 'warren',
       'warring', 'warrior', 'wars', 'warts', 'warwick', 'wary', 'wasabe',
       'wash', 'wash.', 'washed', 'washes', 'washing', 'washingon',
       'washington', 'washout', 'washy', 'wasserman', 'waste', 'wasted',
       'watch', 'watchdog', 'watchers', 'watchful', 'watching',
       'watchmen', 'watchword', 'water', 'waterbrook', 'watercolor',
       'watercress', 'watered', 'waterfall', 'waterford', 'wateriness',
       'watering', 'wateringly', 'watermelon', 'watermelons', 'watermill',
    

Let's try it anyway... We take only the 3000 most common words, thus the strange cases should disappear. However, we might lose some interesting words in the process.

In [20]:
# Utility function to create a list of the 3000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(3000)]

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

In [30]:
# compute the bag of words of the corpus:
nlp = spacy.load('en_core_web_sm') # only the smaller model needed for BoW
nlp.max_length = 24128000 # because len(speeches_corpus) > 1'000'000, == 24127999

# convert corpus in one long string:
wine_corpus_str = " ".join(wine_corpus)


In [31]:
doc_wine = nlp(wine_corpus_str) # tokenize the words
wine_BoW = bag_of_words(doc_wine)
wine_BoW[0:100]

ValueError: [E167] Unknown morphological feature: 'Person' (2313063860588076218). This can happen if the tagger was trained with a different set of morphological features. If you're using a pretrained model, make sure that your models are up to date:
python -m spacy validate

In [ ]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    # Return the most common words:
    return [item[0] for item in Counter(allwords).most_common(2000)]
